# Ejercicio 2

Vamos a explorar como funciona el tokenizador específico que utiliza BERT.

In [ ]:
import os
import certifi
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

d:\git\fundamentos-llms\fund-llms\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carguemos el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Echemos un vistazo a la longitud del vocabulario
vocab_size = len(tokenizer)
print(f"Vocabulario de BERT: {vocab_size:,} tokens")

Vocabulario de BERT: 30,522 tokens


Se pueden tokenizar frases o listas de frases para procesar por lotes o _batches_.

In [109]:
# Vamos a tokenizar un par de oraciones para ver cómo funciona

sentences = [
    "We are the champions, my friends.",
    "The banker is checking my credit application."
]

# Tokenización en lote
batch = tokenizer(sentences, add_special_tokens=True, padding=True, 
                  truncation=True, return_tensors='pt')

print(batch['input_ids'])

tensor([[  101,  2057,  2024,  1996,  3966,  1010,  2026,  2814,  1012,   102],
        [  101,  1996, 13448,  2003,  9361,  2026,  4923,  4646,  1012,   102]])


Volvamos a las frases originales para ver cuál es el procesamiento que hace el tokenizador.

In [6]:
for sequence in batch['input_ids']:
    tokens = tokenizer.convert_ids_to_tokens(sequence)
    print(tokens)

['[CLS]', 'we', 'are', 'the', 'champions', ',', 'my', 'friends', '.', '[SEP]']
['[CLS]', 'the', 'banker', 'is', 'checking', 'my', 'credit', 'application', '.', '[SEP]']


Intentemos con una palabra que no está en el vocabulario de BERT:

In [12]:
word = "Othon"
vocab = tokenizer.get_vocab()

if word in vocab:
    print(f'"{word}" está en el vocabulario.')
else:
    print(f'"{word}" NO está en el vocabulario.')

"Othon" NO está en el vocabulario.


In [ ]:
ood_example = "Othon jumps over the lazy dog."

# Tokenización del ejemplo OOD
ood_tokens = tokenizer(ood_example, add_special_tokens=True, return_tensors='pt')

print(ood_tokens['input_ids'])

ood_sentence = tokenizer.convert_ids_to_tokens(ood_tokens['input_ids'][0])
print(ood_sentence)

tensor([[  101, 27178,  8747, 14523,  2058,  1996, 13971,  3899,  1012,   102]])
['[CLS]', 'ot', '##hon', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '[SEP]']


In [17]:
# Veamos ademas de los input_ids qué otra información nos entrega el tokenizador
print(ood_tokens)

{'input_ids': tensor([[  101, 27178,  8747, 14523,  2058,  1996, 13971,  3899,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Utilizaremos ahora los embeddings para estudiar cómo se codifican las palabras y conceptos.

In [116]:
pet_sentence = "I love my pet python"
coding_sentence = "I love coding in python"

# Tokenización de las oraciones
pet_tokens = tokenizer(pet_sentence, add_special_tokens=True, return_tensors='pt').to(device)
coding_tokens = tokenizer(coding_sentence, add_special_tokens=True, return_tensors='pt').to(device)

In [117]:
# Verificamos los indices de las palabras python en ambas frases
pet_python_index = pet_tokens['input_ids'][0].tolist().index(tokenizer.convert_tokens_to_ids('python')) 
coding_python_index = coding_tokens['input_ids'][0].tolist().index(tokenizer.convert_tokens_to_ids('python'))

print(f"Índice de 'python' en la frase de mascota: {pet_python_index}")
print(f"Índice de 'python' en la frase de programación: {coding_python_index}")

Índice de 'python' en la frase de mascota: 5
Índice de 'python' en la frase de programación: 5


In [118]:
# Ahora obtendremos los embeddings de BERT para estas oraciones

model = BertModel.from_pretrained('bert-base-uncased')

# Mover el modelo a GPU si está disponible
model = model.to(device)

# Pasamos los tokens por el modelo para obtener los embeddings con contexto de las palabras "python"
with torch.no_grad():
    pet_embedding = model(**pet_tokens).last_hidden_state[0, pet_python_index, :].cpu().numpy()
    coding_embedding = model(**coding_tokens).last_hidden_state[0, coding_python_index, :].cpu().numpy()

In [119]:
# Vamos a comparar los embeddings de las dos oraciones con las palabras "snake" y programming"
# No olvidar el token de [CLS] al inicio de las oraciones

with torch.no_grad():
    snake_embedding = model(**tokenizer("snake", return_tensors='pt').to(device)).last_hidden_state[0, 1, :].cpu().numpy()
    programming_embedding = model(**tokenizer("programming", return_tensors='pt').to(device)).last_hidden_state[0, 1, :].cpu().numpy()

In [120]:
model(**tokenizer("snake", return_tensors='pt').to(device)).last_hidden_state.shape

torch.Size([1, 3, 768])

In [ ]:
# Comparar los embeddings que suponemos similares
similarity_pet_snake = cosine_similarity([pet_embedding], [snake_embedding])[0][0]
similarity_coding_programming = cosine_similarity([coding_embedding], [programming_embedding])[0][0]

print(f"Similitud entre 'python' en la frase de mascota y 'snake': {similarity_pet_snake:.4f}")
print(f"Similitud entre 'python' en la frase de programación y 'programming': {similarity_coding_programming:.4f}")

# Ahora comparemos entre embeddings que suponemos no similares
similarity_pet_programming = cosine_similarity([pet_embedding], [programming_embedding])[0][0]
similarity_coding_snake = cosine_similarity([coding_embedding], [snake_embedding])[0][0]

print(f"\nSimilitud entre 'python' en la frase de programación y 'snake': {similarity_coding_snake:.4f}")
print(f"Similitud entre 'python' en la frase de mascota y 'programming': {similarity_pet_programming:.4f}")


Similitud entre 'python' en la frase de mascota y 'snake': 0.6929
Similitud entre 'python' en la frase de programación y 'programming': 0.5615

Similitud entre 'python' en la frase de programación y 'snake': 0.5843
Similitud entre 'python' en la frase de mascota y 'programming': 0.4986
